In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential,load_model
import h5py

# Using ImageDataGenerator instead of Open-Cv as it helps OOM error 

In [13]:
train=ImageDataGenerator(rescale=1/255.0)
test=ImageDataGenerator(rescale=1/255.0)

In [14]:
gen_train=train.flow_from_directory("malaria\\",target_size=(100,100),class_mode="binary")
gen_test=test.flow_from_directory("test-malaria\\",target_size=(100,100),class_mode="binary")

Found 24546 images belonging to 2 classes.
Found 3012 images belonging to 2 classes.


In [15]:
gen_train.class_indices,gen_test.class_indices

({'Parasitized': 0, 'Uninfected': 1}, {'Parasitized': 0, 'Uninfected': 1})

# Model with 2 Conv Layer and 1 Dense Layer

In [16]:
model=Sequential()
model.add(Conv2D(64,(4,4),input_shape=(100,100,3),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Conv2D(64,(4,4),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(1,activation="sigmoid"))

In [17]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [18]:
model.fit_generator(gen_train,validation_data=gen_test,epochs=20)

Epoch 1/20
768/768 [==============================] - 708s 921ms/step - loss: 0.3812 - accuracy: 0.8225 - val_loss: 0.1954 - val_accuracy: 0.9432
Epoch 2/20
768/768 [==============================] - 94s 123ms/step - loss: 0.1620 - accuracy: 0.9473 - val_loss: 0.1685 - val_accuracy: 0.9539
Epoch 3/20
768/768 [==============================] - 93s 120ms/step - loss: 0.1386 - accuracy: 0.9560 - val_loss: 0.1625 - val_accuracy: 0.9502
Epoch 4/20
768/768 [==============================] - 94s 122ms/step - loss: 0.1222 - accuracy: 0.9597 - val_loss: 0.1434 - val_accuracy: 0.9535
Epoch 5/20
768/768 [==============================] - 100s 130ms/step - loss: 0.1069 - accuracy: 0.9646 - val_loss: 0.1793 - val_accuracy: 0.9369
Epoch 6/20
768/768 [==============================] - 95s 124ms/step - loss: 0.0976 - accuracy: 0.9675 - val_loss: 0.1750 - val_accuracy: 0.9459
Epoch 7/20
768/768 [==============================] - 95s 124ms/step - loss: 0.0843 - accuracy: 0.9723 - val_loss: 0.1759 - val_

In [19]:
model.save("malaria_model.h5")

In [20]:
model=load_model("malaria_model.h5")

In [21]:
gen_test.class_indices,gen_train.class_indices

({'Parasitized': 0, 'Uninfected': 1}, {'Parasitized': 0, 'Uninfected': 1})

In [23]:
import glob
filepath="test-malaria\\Uninfected\\C39P4thinF_original_IMG_20150622_111206_cell_75.png"

In [24]:
from keras.models import load_model
import cv2
import numpy as np

def preprocessing(i):
    data=cv2.imread(i)
    data=cv2.resize(data,(100,100))
    data=np.array(data)
    data=data.reshape(-1,100,100,3)
    return data/255.0

model=load_model("malaria_model.h5")
data=preprocessing(filepath)
s=model.predict_proba(data)
p=model.predict_classes(data)[0][0]

if p==0:
    print("The cell is parasitized and the probability is :",1-s[0][0])
else :
    print("The cell is uninfected and probability is :",s[0][0])

The cell is uninfected and probability is : 0.85143065
